#Install

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import random
warnings.filterwarnings("ignore")
import torch
from torch import nn
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler

In [2]:
# !pip install xgboost
# !pip install pycaret[full]

In [3]:
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score

#Data Processing

In [4]:
device='cpu' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(0)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
base_path = '/content/drive/MyDrive/2022_Unithon_Data/data/dataset'

In [7]:
# 파일 불러오기
def dataframe_from_csv(target):
  return pd.read_csv(target).rename(columns=lambda x:x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

train_files = sorted([x for x in Path(base_path+'/train/').glob('*.csv')])
val_files = sorted([x for x in Path(base_path+'/val/').glob('*.csv')])

train = dataframe_from_csvs(train_files)
val = dataframe_from_csvs(val_files)
test = pd.read_csv(base_path+'/test.csv')
print(f'train: {len(train)}')
print(f'validation: {len(val)}')
print(f'test: {len(test)}')

train: 62564
validation: 7820
test: 7820


In [8]:
train = train.sample(frac=1).reset_index(drop=True)
train

,site,sid,leaktype,C01,C02,C03,C04,C05,C06,C07,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,S-4772025022,S-0359369084109814,out,4,63,7,7,9,13,9,...,7,7,7,7,7,9,9,11,7,420
1,S-2711010100,S-0359369084032917,normal,4,9,6,5,6,6,6,...,5,6,6,6,6,5,6,7,6,260
2,S-4721011100,S-0359369084142864,out,6,277,34,34,43,9,15,...,2,2,2,4,2,2,2,2,2,350
3,S-2911010107,S-0359369084041454,normal,4,6,5,5,6,5,5,...,6,6,7,6,5,6,6,5,6,250
4,S-2911010107,S-0359369084047311,normal,4,17,5,5,5,5,5,...,4,5,5,5,5,9,9,5,5,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62559,S-4677025028,S-0359369084263413,in,5,74,11,13,16,11,8,...,3,5,11,5,5,5,5,5,8,370
62560,S-4571025029,S-0359369084037767,normal,5,7,7,6,5,5,5,...,6,6,5,6,5,6,7,5,6,250
62561,S-4677025033,S-0359369083954723,out,4,18,4,5,5,5,5,...,5,5,4,5,4,5,5,5,7,400
62562,S-4772025022,S-0359369084107743,out,2,367,9,16,9,5,3,...,3,3,3,5,3,3,3,5,3,300


In [9]:
train = train.drop(['site', 'sid'], axis=1)
val = val.drop(['site', 'sid'], axis=1)
test = test.drop(['site', 'sid'], axis=1)
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

# train을 target과 feature로 나눠줍니다.
train_x=train.drop(['leaktype'], axis=1)
train_y=train['leaktype']
val_x=val.drop(['leaktype'], axis=1)
val_y=val['leaktype']
test_x=test.drop(['leaktype'], axis=1)
test_y=test['leaktype']

In [10]:
print(train_x)
print(train_y)

       C01  C02  C03  C04  C05  C06  C07  C08  C09  C10  ...  C17  C18  C19  \
0        4   63    7    7    9   13    9    9    7    9  ...    7    7    7   
1        4    9    6    5    6    6    6    4    6    5  ...    5    6    6   
2        6  277   34   34   43    9   15   15   15    9  ...    2    2    2   
3        4    6    5    5    6    5    5    5    5    6  ...    6    6    7   
4        4   17    5    5    5    5    5    5    5    4  ...    4    5    5   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
62559    5   74   11   13   16   11    8    8   11   18  ...    3    5   11   
62560    5    7    7    6    5    5    5    5    5    6  ...    6    6    5   
62561    4   18    4    5    5    5    5    4    4    5  ...    5    5    4   
62562    2  367    9   16    9    5    3    3    3    3  ...    3    3    3   
62563    4  121    7    9   21    5    5    6    6    4  ...    4    4    4   

       C20  C21  C22  C23  C24  C25  C26  
0       

In [11]:
type(train_x)

pandas.core.frame.DataFrame

In [12]:
sc=StandardScaler()

LightGBM

In [13]:
train_x=sc.fit_transform(train_x)
val_x=sc.fit_transform(val_x)
test_x=sc.fit_transform(test_x)

#Train

In [111]:
d_train=lgb.Dataset(train_x,label=train_y)
d_val = lgb.Dataset(val_x,label=val_y)
params={}
#고정
params['boosting_type']='gbdt'
params['objective']='multiclass'
params['metric']='multi_logloss'
params['num_class']=5

#수정가능
params['seed'] = 0
params['min_data_in_leaf'] = 10
params['bagging_fraction'] = 0.3
params['lambda'] = 0.3
params['feature_fraction'] = 0.8
params['max_bin'] = 255 #default = 255
params['max_depth']=20
params['learning_rate']=0.01
params['num_leaves']=1000

In [112]:
clf=lgb.train(params, d_train,5000, valid_sets=[d_train, d_val], verbose_eval=True )# epoch   

[1]	training's multi_logloss: 1.48581	valid_1's multi_logloss: 1.48931
[2]	training's multi_logloss: 1.47435	valid_1's multi_logloss: 1.48094
[3]	training's multi_logloss: 1.46329	valid_1's multi_logloss: 1.47289
[4]	training's multi_logloss: 1.45247	valid_1's multi_logloss: 1.46519
[5]	training's multi_logloss: 1.44181	valid_1's multi_logloss: 1.45739
[6]	training's multi_logloss: 1.43152	valid_1's multi_logloss: 1.4502
[7]	training's multi_logloss: 1.42168	valid_1's multi_logloss: 1.44285
[8]	training's multi_logloss: 1.41116	valid_1's multi_logloss: 1.43526
[9]	training's multi_logloss: 1.40142	valid_1's multi_logloss: 1.4283
[10]	training's multi_logloss: 1.39232	valid_1's multi_logloss: 1.42169
[11]	training's multi_logloss: 1.38278	valid_1's multi_logloss: 1.41487
[12]	training's multi_logloss: 1.37305	valid_1's multi_logloss: 1.40788
[13]	training's multi_logloss: 1.36414	valid_1's multi_logloss: 1.4014
[14]	training's multi_logloss: 1.35508	valid_1's multi_logloss: 1.39498
[15]

KeyboardInterrupt: ignored

#Validate

In [106]:
#Prediction
y_pred=clf.predict(val_x)
y_pred=[np.argmax(line) for line in y_pred]

In [107]:
print("Validation F1 score: ", f1_score(val_y, y_pred, average='macro'))

Validation F1 score:  0.6725849880715724


In [108]:
print("Validation Accuracy Score: ", accuracy_score(val_y, y_pred))

Validation Accuracy Score:  0.7359335038363172
